# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df = pd.read_csv("assets/nhl.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
nhl_df = nhl_df[(nhl_df['year'] == 2018)]
cities = cities.iloc[:-1,[0,3,5,6,7,8]]
cities['Population 2016 est.'] = cities['Population (2016 est.)[8]'].str.strip().str.replace(',', '').astype(int)
#global cities
cities = cities.reset_index()

def nhl_correlation(): 
    # YOUR CODE HERE
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    #global cities 
    NHL_cities = cities[['Metropolitan area','Population (2016 est.)[8]', 'NHL']]
    column_names = ['Region','Population 2016 est.', 'Team']
    NHL_cities.columns = column_names
    
    NHL_cities['Team'] = NHL_cities['Team'].str.replace(r'\[note\s+\d+\]','', regex=True)
    NHL_cities['Team'] = NHL_cities['Team'].str.strip().str.replace(r'[^a-zA-Z\s]+', '', regex=True).str.strip()
    NHL_cities['Team'] = NHL_cities['Team'].str.strip().str.replace('Kings Ducks', 'Kings, Ducks')
    NHL_cities['Team'] = NHL_cities['Team'].str.strip().str.replace('Rangers Islanders Devils', 'Rangers, Islanders, Devils')
    NHL_city_df = NHL_cities.assign(Team = NHL_cities['Team'].str.split(',')).explode('Team')
    NHL_city_df['Team'] = NHL_city_df['Team'].str.strip()
    NHL_city_df.reset_index()
    
    NHL_city_df = NHL_city_df[(NHL_city_df['Team'] != ' ') &
        (NHL_city_df['Team'] != '')]
                   
    #print(NHL_city_df)
    #print(len(NHL_city_df))
    
    nhl_wl_df = nhl_df[~(nhl_df['team'].str.contains(
        'Division'))][['team', 'W', 'L']].reset_index()
    #print(nhl_wl_df)
    nhl_wl_df['WL'] = nhl_wl_df['W'].str.strip().astype(int)/ (nhl_wl_df['W'].str.strip().astype(int) + nhl_wl_df['L'].str.strip().astype(int))
    
    # Split the 'LocationTeam' column into 'Region' and 'Team'
    nhl_wl_df['team'] = nhl_wl_df['team'].str.replace(r'*','')
    nhl_wl_df[['Region', 'Team']] = nhl_wl_df['team'].str.rsplit(n=1, expand=True)
    nhl_wl_df = nhl_wl_df[['Team', 'W', 'L', 'WL']]
    nhl_wl_df['Team'] = nhl_wl_df['Team'].str.replace('Leafs', 'Maple Leafs')
    nhl_wl_df['Team'] = nhl_wl_df['Team'].str.replace('Wings', 'Red Wings')
    nhl_wl_df['Team'] = nhl_wl_df['Team'].str.replace('Jackets', 'Blue Jackets')
    nhl_wl_df['Team'] = nhl_wl_df['Team'].str.replace('Knights', 'Golden Knights')
    
    #print(nhl_wl_df)
    
    nhl_merge_df = pd.merge(nhl_wl_df, NHL_city_df, how = "inner", on = "Team")
  
    #print(nhl_merge_df)
    
    #print(nhl_wl_df)
    #print(len(nhl_wl_df))
    
    agg_func = ["mean"]
    agg_df = nhl_merge_df.groupby(['Region', 'Population 2016 est.'])['WL'].agg(agg_func)
    print(agg_df)
    print(len(agg_df))
    agg_df = agg_df.reset_index()
    
    agg_columns = ['Region','Population by Region', 'Win Loss by Region']
    agg_df.columns = agg_columns

    population_by_region = agg_df['Population by Region'].astype(int)
    win_loss_by_region = agg_df['Win Loss by Region']
    
    #print(agg_df)
    #print(len(agg_df))
    corr, pval = stats.pearsonr(
        population_by_region, win_loss_by_region)
    return corr
nhl_correlation()

                                                 mean
Region                 Population 2016 est.          
Boston                 4794447               0.714286
Buffalo                1132804               0.357143
Calgary                1392609               0.513889
Chicago                9512999               0.458333
Columbus               2041520               0.600000
Dallas–Fort Worth      7233323               0.567568
Denver                 2853077               0.589041
Detroit                4297617               0.434783
Edmonton               1321426               0.473684
Las Vegas              2155664               0.680000
Los Angeles            13310447              0.622895
Miami–Fort Lauderdale  6066387               0.594595
Minneapolis–Saint Paul 3551036               0.633803
Montreal               4098927               0.420290
Nashville              1865298               0.746479
New York City          20153634              0.518201
Ottawa                 13237

0.012486162921209909

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
nba_df = nba_df[(nba_df['year'] == 2018)]
#print(nba_df)
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
#print(cities)

def nba_correlation():
    # YOUR CODE HERE
    NBA_cities = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NBA']]
    column_names = ['Region','Population 2016 est.', 'Team']
    NBA_cities.columns = column_names
    #NBA_cities['Team'] = NBA_cities['Team'].str.replace(r'\[note\s+\d+\]','', regex=True)
    #NBA_cities['Team'] =  NBA_cities['Team'].str.replace(r'(?<!\w)\[note\s+\d+\](?!\w)', '', regex = True)
    NBA_cities['Team'] =  NBA_cities['Team'].str.replace(r'\[note\s+\d+\](?!\w)', '', regex = True)
    NBA_cities['Team'] = NBA_cities['Team'].str.replace(r'[^\w\s]+', '', regex=True).str.strip()
    NBA_cities = NBA_cities[(NBA_cities['Team'] != ' ') &
        (NBA_cities['Team'] != '')]
    #df['Team'] = df['Team'].str.replace('Nets Knicks', 'Nets, Knicks')
    NBA_cities['Team'] = NBA_cities['Team'].str.strip().str.replace('Knicks Nets', 'Knicks, Nets')
    NBA_cities['Team'] = NBA_cities['Team'].str.strip().str.replace('Lakers Clippers', 'Lakers, Clippers')
    
    #print(NBA_cities)
    
    NBA_city_df = NBA_cities.assign(Team = NBA_cities['Team'].str.split(',')).explode('Team')
    NBA_city_df.reset_index()
    NBA_city_df = NBA_city_df[['Team', 'Region', 'Population 2016 est.']]
    NBA_city_df['Team'] = NBA_city_df['Team'].str.strip()
    #print(NBA_city_df)
    #print(len(NBA_city_df))
    
    
    #print(len(NBA_cities))
    #print(NBA_cities)
    #raise NotImplementedError()
    nba_wl_df = nba_df[~(nba_df['team'].str.contains(
        'Division'))][['team', 'W', 'L']].reset_index()
    #print(nba_wl_df['W'])
    nba_wl_df['WL %'] = nba_wl_df['W'].str.strip().astype(int)/ (nba_wl_df['W'].str.strip().astype(int) + nba_wl_df['L'].str.strip().astype(int))
    nba_wl_df['team'] = nba_wl_df['team'].str.replace(r'*','')
    nba_wl_df['team'] = nba_wl_df['team'].str.replace(r'\(\d+\)', '', regex = True)
    

    # Split the 'LocationTeam' column into 'Region' and 'Team'
    nba_wl_df[['Region', 'Team']] = nba_wl_df['team'].str.rsplit(n=1, expand=True)
    nba_wl_df['Team'] = nba_wl_df['Team'].str.replace(r'Blazers','Trail Blazers', regex = True)
    nba_wl_df['Region'] = nba_wl_df['Region'].str.replace(r'Portland Trail','Portland', regex = True)
    #San Francisco Bay Area
    nba_wl_df['Region'] = nba_wl_df['Region'].str.replace(r'Golden State','San Francisco Bay Area', regex = True)
    nba_wl_df['Region'] = nba_wl_df['Region'].str.replace(r'Washington','Washington, D.C.', regex = True)
    nba_wl_df['Region'] = nba_wl_df['Region'].str.replace(r'New York','New York City', regex = True)
    nba_wl_df['Region'] = nba_wl_df['Region'].str.replace(r'Brooklyn','New York City', regex = True)
    nba_wl_df['Region'] = nba_wl_df['Region'].str.replace(r'Miami', 'Miami–Fort Lauderdale', regex = True)
    nba_wl_df = nba_wl_df[['Team', 'W', 'L', 'WL %']]
    #Washington, D.C.
    #New York City
    #Miami–Fort Lauderdale
    #print(nba_wl_df)
    #print(len(nba_wl_df))
    
    NBA_merge_df = pd.merge(nba_wl_df, NBA_city_df, how = "inner", on = "Team")
    #print(NBA_merge_df)
    
    agg_func = ["mean"]
    agg_df = NBA_merge_df.groupby(['Region', 'Population 2016 est.'])['WL %'].agg(agg_func)
    #print(agg_df)
    #print(len(agg_df))
    agg_df = agg_df.reset_index()
    
    agg_columns = ['Region','Population by Region', 'Win Loss by Region']
    agg_df.columns = agg_columns

    population_by_region = agg_df['Population by Region'].astype(int)
    win_loss_by_region = agg_df['Win Loss by Region']
    
    #print(agg_df)
    #print(len(agg_df))
    
    #population_by_region = [] # pass in metropolitan area population from cities
    #win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    
    corr, pval = stats.pearsonr(
        population_by_region, win_loss_by_region)

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    
    return corr
    #return stats.pearsonr(population_by_region, win_loss_by_region)
nba_correlation()

-0.17657160252844614

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [9]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
mlb_df = mlb_df[(mlb_df['year'] == 2018)]
#print(mlb_df)

cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
#print(cities)

def mlb_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()

    mlb_cities = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'MLB']]
    column_names = ['Region','Population 2016 est.', 'Team']
    mlb_cities.columns = column_names
    mlb_cities['Population 2016 est.'] = mlb_cities['Population 2016 est.'].astype(int)
    mlb_cities['Team'] =  mlb_cities['Team'].str.replace(r'\[note\s+\d+\](?!\w)', '', regex = True)
    mlb_cities['Team'] = mlb_cities['Team'].str.replace(r'[^\w\s]+', '')
    mlb_cities = mlb_cities[(mlb_cities['Team'] != ' ') & (mlb_cities['Team'] != '')]
    mlb_cities['Team'] = mlb_cities['Team'].str.replace('Yankees Mets', 'Yankees, Mets')
    mlb_cities['Team'] = mlb_cities['Team'].str.replace('Dodgers Angels', 'Dodgers, Angels')
    mlb_cities['Team'] = mlb_cities['Team'].str.replace('Giants Athletics', 'Giants, Athletics')
    mlb_cities['Team'] = mlb_cities['Team'].str.replace('Cubs White Sox', 'Cubs, White Sox')
    #NFL_city_df = NFL_cities.assign(Team = NFL_cities['Team'].str.split(',')).explode('Team')
    MLB_city_df = mlb_cities.assign(Team = mlb_cities['Team'].str.split(',')).explode('Team')
    MLB_city_df['Team'] = MLB_city_df['Team'].str.strip()
    #print(MLB_city_df)
    #print(len(MLB_city_df))
    
    mlb_wl_df = mlb_df[['team', 'W', 'L','W-L%']].reset_index()
    mlb_wl_df[['Region', 'Team']] = mlb_wl_df['team'].str.split(n=1, expand=True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'New', 'New York', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'York Yankees', 'Yankees', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'City Royals', 'Royals', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'Louis Cardinals', 'Cardinals', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'Angeles Angels', 'Angels', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'Angeles Dodgers', 'Dodgers', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'Francisco Giants', 'Giants', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'Diego Padres', 'Padres', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'York Mets', 'Mets', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'Bay Rays', 'Rays', regex = True)
    
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'New York', 'New York City', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'San', 'San Diego', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'St.', 'St. Lous', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'Los', 'Los Angeles', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'Kansas', 'Kansas City', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'Arizona', 'Phoenix', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'Colorado', 'Denver', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'Washington', 'Washington D.C.', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'Texas', 'Dallas–Fort Worth', regex = True)
    mlb_wl_df['WL %'] = mlb_wl_df['W'].astype(int)/(mlb_wl_df['W'].astype(int)+mlb_wl_df['L'].astype(int))
    mlb_wl_df = mlb_wl_df[['Team', 'W', 'L', 'WL %']]
    #print(mlb_wl_df)
    #print(len(mlb_wl_df))
    
    MLB_merge_df = pd.merge(mlb_wl_df, MLB_city_df, how = "inner", on = "Team")
    #print(MLB_merge_df)
    
    agg_func = ["mean"]
    agg_df = MLB_merge_df.groupby(['Region', 'Population 2016 est.'])['WL %'].agg(agg_func)
    agg_df = agg_df.reset_index()
    
    agg_columns = ['Region','Population by Region', 'Win Loss by Region']
    agg_df.columns = agg_columns
    
    
    #print(agg_df)
    #print(len(agg_df))
    population_by_region = agg_df['Population by Region']
    win_loss_by_region = agg_df['Win Loss by Region']
    
    corr, pval = stats.pearsonr(
        population_by_region, win_loss_by_region)
  
    #population_by_region = [] # pass in metropolitan area population from cities
    #win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    return corr
mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [11]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df = pd.read_csv("assets/nfl.csv")
nfl_df = nfl_df[(nfl_df['year'] == 2018)]
#print(nfl_df)

cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1,[0,3,5,6,7,8]]
#print(cities)


def nfl_correlation(): 
    # YOUR CODE HERE
    NFL_cities = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NFL']]
    column_names = ['Region','Population 2016 est.', 'Team']
    NFL_cities.columns = column_names
    NFL_cities['Team'] = NFL_cities['Team'].str.replace(r'\[note\s+\d+\](?!\w)','', regex = True)
    NFL_cities['Team'] = NFL_cities['Team'].str.replace(r'[^\w\s]+', '')
    NFL_cities = NFL_cities[(NFL_cities['Team'] != ' ') & (NFL_cities['Team'] != '')]
    NFL_cities['Team'] = NFL_cities['Team'].str.replace('Giants Jets', 'Giants, Jets')
    NFL_cities['Team'] = NFL_cities['Team'].str.replace('Rams Chargers', 'Rams, Chargers')
    NFL_cities['Team'] = NFL_cities['Team'].str.replace('49ers Raiders', '49ers,Raiders')
    NFL_cities['Region'] = NFL_cities['Region'].str.replace('Tampa Bay Area','Tampa Bay')
    NFL_cities['Region'] = NFL_cities['Region'].str.replace('San Francisco Bay Area','San Francisco')
    NFL_city_df = NFL_cities.assign(Team = NFL_cities['Team'].str.split(',')).explode('Team')
    NFL_city_df['Team'] = NFL_city_df['Team'].str.strip()
   
    ##NFL_city_df['Team'] = NFL_city_df['].str.split()
    NFL_city_df['Population 2016 est.'] = NFL_city_df['Population 2016 est.'].astype(int)
    
    NFL_wl_df = nfl_df[~(nfl_df['team'].str.contains('AFC'))][['team','W', 'W-L%']]
    NFL_wl_df = NFL_wl_df[~(NFL_wl_df['team'].str.contains('NFC'))][['team','W', 'W-L%']]
    NFL_wl_df['team'] = NFL_wl_df['team'].str.replace(r'[+*]$', '', regex = True)
    NFL_wl_df[['Region', 'Team']] = NFL_wl_df['team'].str.rsplit(n=1, expand=True)
    NFL_wl_df['L'] = ( NFL_wl_df['W'].astype(int) - (NFL_wl_df['W-L%'].astype(float) * NFL_wl_df['W'].astype(int)))/NFL_wl_df['W-L%'].astype(float)
    NFL_wl_df['L'] = NFL_wl_df['L'].round(0).astype(int)
    NFL_wl_df['WL'] = NFL_wl_df['W'].astype(int)/( NFL_wl_df['W'].astype(int)+ NFL_wl_df['L'].astype(int))
    NFL_wl_df = NFL_wl_df[['Team','WL','Region']]
    NFL_wl_df['Region'] = NFL_wl_df['Region'].str.replace('New England','Boston')
    NFL_wl_df['Region'] = NFL_wl_df['Region'].str.replace('New York','New York City')
    NFL_wl_df['Region'] = NFL_wl_df['Region'].str.replace('Tenessee','Nashville')
    NFL_wl_df['Region'] = NFL_wl_df['Region'].str.replace('Arizona','Phoenix')
    NFL_wl_df['Region'] = NFL_wl_df['Region'].str.replace('Carolina','Charlotte')
    NFL_wl_df['Region'] = NFL_wl_df['Region'].str.replace('Minnesota','Minneapolis–Saint Paul')
    NFL_wl_df['Region'] = NFL_wl_df['Region'].str.strip()
    NFL_wl_df['Team'] = NFL_wl_df['Team'].str.strip()
    NFL_wl_df = NFL_wl_df[['Team', 'WL']]
    #print(NFL_wl_df)
    #print(len(NFL_wl_df))
    
    #MLB_merge_df = pd.merge(mlb_wl_df, MLB_city_df, how = "inner", on = "Team")
    #print(MLB_merge_df)
    NFL_merge_df = pd.merge(NFL_wl_df, NFL_city_df, how = "inner", on = "Team")
    #print(NFL_merge_df)
    
    agg_func = ["mean"]
    agg_df = NFL_merge_df.groupby(['Region', 'Population 2016 est.'])['WL'].agg(agg_func)
    agg_df = agg_df.reset_index()
    
    agg_columns = ['Region','Population by Region', 'Win Loss by Region']
    agg_df.columns = agg_columns
    
    #print(agg_df)
    #print(len(agg_df))
    population_by_region = agg_df['Population by Region']   # pass in metropolitan area population from cities
    win_loss_by_region = agg_df['Win Loss by Region']       # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    
    corr, pval = stats.pearsonr(
        population_by_region, win_loss_by_region)
     
    
    #population_by_region = [] # pass in metropolitan area population from cities
    #win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    #return stats.pearsonr(population_by_region, win_loss_by_region)
    return corr
nfl_correlation()

0.004922112149349409

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [12]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from scipy.stats import ttest_rel
from scipy.stats import ttest_ind

##pd.set_option("display.max_rows", None, "display.max_columns", None)
mlb_df = pd.read_csv("assets/mlb.csv")
mlb_df = mlb_df[(mlb_df['year'] == 2018)]
#print(mlb_df)

nhl_df = pd.read_csv("assets/nhl.csv")
nhl_df = nhl_df[(nhl_df['year'] == 2018)]
#print(nhl_df)

nba_df=pd.read_csv("assets/nba.csv")
nba_df = nba_df[(nba_df['year'] == 2018)]
#print(nba_df)

nfl_df=pd.read_csv("assets/nfl.csv")
nfl_df = nfl_df[(nfl_df['year'] == 2018)]
#print(nfl_df)


cities=pd.read_html("assets/wikipedia_data.html")[1]

cities=cities.iloc[:-1,[0,3,5,6,7,8]]
#print(cities)

def sports_team_performance():
    # YOUR CODE HERE
    #cities_df = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NFL']]
    #column_names = ['Region','Population 2016 est.', 'Team']
    #cities_df.columns = column_names
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    sport_cities = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'MLB', 'NHL','NBA', 'NFL' ]]
    column_names = ['Region','Population 2016 est.', 'MLB', 'NHL','NBA', 'NFL']

    sport_cities.columns = column_names
    
    sport_cities['Region'] = sport_cities['Region'].str.replace('San Francisco Bay Area','San Francisco')
    sport_cities['Region'] = sport_cities['Region'].str.replace('Tampa Bay Area','Tampa Bay')
    
    print('*****************************************************************************************************')
    print('*********************************************MLB*****************************************************')
    #Baseball Teams
    mlb_wl_df = mlb_df[['team', 'W', 'L','W-L%']].reset_index()
    mlb_wl_df[['Region', 'Team']] = mlb_wl_df['team'].str.split(n=1, expand=True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'New', 'New York', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'York Yankees', 'Yankees', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'City Royals', 'Royals', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'Louis Cardinals', 'Cardinals', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'Angeles Angels', 'Angels', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'Angeles Dodgers', 'Dodgers', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'Francisco Giants', 'Giants', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'Diego Padres', 'Padres', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'York Mets', 'Mets', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'Bay Rays', 'Rays', regex = True)
    
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'New York', 'New York City', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'San', 'San Diego', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'St.', 'St. Lous', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'Los', 'Los Angeles', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'Kansas', 'Kansas City', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'Arizona', 'Phoenix', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'Colorado', 'Denver', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'Washington', 'Washington D.C.', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'Texas', 'Dallas–Fort Worth', regex = True)
    mlb_wl_df['WL'] = mlb_wl_df['W'].astype(int)/(mlb_wl_df['W'].astype(int)+mlb_wl_df['L'].astype(int))
    mlb_wl_df = mlb_wl_df[['Team', 'W', 'L', 'WL']]
    mlb_wl_df.rename(columns={'Team': 'MLB_Team'}, inplace=True)
    
    sport_cities['Population 2016 est.'] = sport_cities['Population 2016 est.'].astype(int)
    sport_cities['MLB'] =  sport_cities['MLB'].str.replace(r'\[note\s+\d+\](?!\w)', '', regex = True)
    sport_cities['MLB'] = sport_cities['MLB'].str.replace(r'[^\w\s]+', '')
    sport_cities['MLB'] = sport_cities['MLB'].str.replace('Yankees Mets', 'Yankees, Mets')
    sport_cities['MLB'] = sport_cities['MLB'].str.replace('Dodgers Angels', 'Dodgers, Angels')
    sport_cities['MLB'] = sport_cities['MLB'].str.replace('Giants Athletics', 'Giants, Athletics')
    sport_cities['MLB'] = sport_cities['MLB'].str.replace('Cubs White Sox', 'Cubs, White Sox')
    
    MLB_city_df = sport_cities.assign(MLB = sport_cities['MLB'].str.split(',')).explode('MLB')
    MLB_city_df = MLB_city_df[['Region','Population 2016 est.', 'MLB']]
    MLB_city_df = MLB_city_df[(MLB_city_df['MLB'] != ' ') & (MLB_city_df['MLB'] != '')]
   
    MLB_city_df['MLB'] = MLB_city_df['MLB'].str.strip()
    MLB_city_df.rename(columns={'MLB': 'MLB_Team'}, inplace=True)
    #print(MLB_city_df)
    #print(len(MLB_city_df)) 
    
    #Baseball Teams
    mlb_wl_df = mlb_df[['team', 'W', 'L','W-L%']].reset_index()
    mlb_wl_df[['Region', 'Team']] = mlb_wl_df['team'].str.split(n=1, expand=True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'New', 'New York', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'York Yankees', 'Yankees', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'City Royals', 'Royals', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'Louis Cardinals', 'Cardinals', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'Angeles Angels', 'Angels', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'Angeles Dodgers', 'Dodgers', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'Francisco Giants', 'Giants', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'Diego Padres', 'Padres', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'York Mets', 'Mets', regex = True)
    mlb_wl_df['Team'] = mlb_wl_df['Team'].str.replace(r'Bay Rays', 'Rays', regex = True)
    
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'New York', 'New York City', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'San', 'San Diego', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'St.', 'St. Lous', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'Los', 'Los Angeles', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'Kansas', 'Kansas City', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'Arizona', 'Phoenix', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'Colorado', 'Denver', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'Washington', 'Washington D.C.', regex = True)
    mlb_wl_df['Region'] = mlb_wl_df['Region'].str.replace(r'Texas', 'Dallas–Fort Worth', regex = True)
    mlb_wl_df['WL'] = mlb_wl_df['W'].astype(int)/(mlb_wl_df['W'].astype(int)+mlb_wl_df['L'].astype(int))
    mlb_wl_df = mlb_wl_df[['Team', 'W', 'L', 'WL']]
    mlb_wl_df.rename(columns={'Team': 'MLB_Team'}, inplace=True)
    
    MLB_merge_df = pd.merge(mlb_wl_df, MLB_city_df, how = "inner", on = "MLB_Team")
    #print(MLB_merge_df)
    #print(len(MLB_merge_df))
    
    mlb_agg_func = ["mean"]
    mlb_agg_df = MLB_merge_df.groupby(['Region', 'Population 2016 est.'])['WL'].agg(mlb_agg_func)
    print(mlb_agg_df)
    print(len(mlb_agg_df))
    mlb_agg_df = mlb_agg_df.reset_index()
    
    mlb_agg_columns = ['Region','Population by Region', 'Win Loss by Region']
    mlb_agg_df.columns = mlb_agg_columns
      
    ###MLB/NHL
    ### MLB/NHL 17
    print('****************************************************NHL**********************************************')
    #Hockey Teams
    nhl_wl_df = nhl_df[~(nhl_df['team'].str.contains(
        'Division'))][['team', 'W', 'L']].reset_index()
    #print(nhl_wl_df)
    nhl_wl_df['WL'] = nhl_wl_df['W'].str.strip().astype(int)/ (nhl_wl_df['W'].str.strip().astype(int) + nhl_wl_df['L'].str.strip().astype(int))
    
    # Split the 'LocationTeam' column into 'Region' and 'Team'
    nhl_wl_df['team'] = nhl_wl_df['team'].str.replace(r'*','')
    nhl_wl_df[['Region', 'Team']] = nhl_wl_df['team'].str.rsplit(n=1, expand=True)
    nhl_wl_df = nhl_wl_df[['Team', 'W', 'L', 'WL']]
    nhl_wl_df['Team'] = nhl_wl_df['Team'].str.replace('Leafs', 'Maple Leafs')
    nhl_wl_df['Team'] = nhl_wl_df['Team'].str.replace('Wings', 'Red Wings')
    nhl_wl_df['Team'] = nhl_wl_df['Team'].str.replace('Jackets', 'Blue Jackets')
    nhl_wl_df['Team'] = nhl_wl_df['Team'].str.replace('Knights', 'Golden Knights')
    nhl_wl_df.rename(columns={'Team': 'NHL_Team'}, inplace=True)
    
    sport_cities['NHL'] = sport_cities['NHL'].str.replace(r'\[note\s+\d+\]','', regex = True)
    sport_cities['NHL'] = sport_cities['NHL'].str.strip().astype(str).str.replace(r'[^a-zA-Z\s]+', '', regex=True).str.strip()
    sport_cities['NHL'] = sport_cities['NHL'].str.strip().str.replace('Kings Ducks', 'Kings, Ducks')
    sport_cities['NHL'] = sport_cities['NHL'].str.strip().str.replace('Rangers Islanders Devils', 'Rangers, Islanders, Devils')
    NHL_city_df = sport_cities.assign(NHL = sport_cities['NHL'].str.split(',')).explode('NHL')
    NHL_city_df['NHL'] = NHL_city_df['NHL'].str.strip()
    NHL_city_df.reset_index()
    
    NHL_city_df = NHL_city_df[['Region','Population 2016 est.', 'NHL']]
    NHL_city_df = NHL_city_df[(NHL_city_df['NHL'] != ' ') &
        (NHL_city_df['NHL'] != '')]
    
    NHL_city_df.rename(columns={'NHL': 'NHL_Team'}, inplace=True)
    
     #print(NHL_city_df)
    #print(len(NHL_city_df))
    ### NHL/NHL 28
    
    nhl_merge_df = pd.merge(nhl_wl_df, NHL_city_df, how = "inner", on = "NHL_Team")
  
    #print(nhl_merge_df)
    
    #print(nhl_wl_df)
    #print(len(nhl_wl_df))
    
    nhl_agg_func = ["mean"]
    nhl_agg_df = nhl_merge_df.groupby(['Region', 'Population 2016 est.'])['WL'].agg(nhl_agg_func)
    print(nhl_agg_df)
    print(len(nhl_agg_df))
    nhl_agg_df = nhl_agg_df.reset_index()

    nhl_agg_columns = ['Region','Population by Region', 'Win Loss by Region']
    nhl_agg_df.columns = nhl_agg_columns
   
    #NBA Teams
    print("*********************************************NBA******************************************************")
    nba_wl_df = nba_df[~(nba_df['team'].str.contains(
        'Division'))][['team', 'W', 'L']].reset_index()
    #print(nba_wl_df['W'])
    nba_wl_df['WL'] = nba_wl_df['W'].str.strip().astype(int)/ (nba_wl_df['W'].str.strip().astype(int) + nba_wl_df['L'].str.strip().astype(int))
    nba_wl_df['team'] = nba_wl_df['team'].str.replace(r'*','')
    nba_wl_df['team'] = nba_wl_df['team'].str.replace(r'\(\d+\)', '', regex = True)
    
    # Split the 'LocationTeam' column into 'Region' and 'Team'
    nba_wl_df[['Region', 'Team']] = nba_wl_df['team'].str.rsplit(n=1, expand=True)
    nba_wl_df['Team'] = nba_wl_df['Team'].str.replace(r'Blazers','Trail Blazers', regex = True)
    nba_wl_df['Region'] = nba_wl_df['Region'].str.replace(r'Portland Trail','Portland', regex = True)
    #San Francisco Bay Area
    nba_wl_df['Region'] = nba_wl_df['Region'].str.replace(r'Golden State','San Francisco', regex = True)
    nba_wl_df['Region'] = nba_wl_df['Region'].str.replace(r'Washington','Washington, D.C.', regex = True)
    nba_wl_df['Region'] = nba_wl_df['Region'].str.replace(r'New York','New York City', regex = True)
    nba_wl_df['Region'] = nba_wl_df['Region'].str.replace(r'Brooklyn','New York City', regex = True)
    nba_wl_df['Region'] = nba_wl_df['Region'].str.replace(r'Miami', 'Miami–Fort Lauderdale', regex = True)
    nba_wl_df = nba_wl_df[['Team', 'W', 'L', 'WL']]
    nba_wl_df.rename(columns={'Team': 'NBA_Team'}, inplace=True)
    #print(nba_wl_df)
    
    sport_cities['NBA'] =  sport_cities['NBA'].str.replace(r'\[note\s+\d+\](?!\w)', '', regex = True)
    sport_cities['NBA'] = sport_cities['NBA'].str.replace(r'[^\w\s]+', '', regex=True).str.strip()
    #sport_cities = sport_cities[(sport_cities['NBA'] != ' ') &
    #    (sport_cities['NBA'] != '')]
    
    sport_cities['NBA'] = sport_cities['NBA'].str.strip().str.replace('Knicks Nets', 'Knicks, Nets')
    sport_cities['NBA'] = sport_cities['NBA'].str.strip().str.replace('Lakers Clippers', 'Lakers, Clippers')
    
    NBA_city_df = sport_cities.assign(NBA = sport_cities['NBA'].str.split(',')).explode('NBA')
    NBA_city_df.reset_index()
    
    NBA_city_df = NBA_city_df[['NBA', 'Region', 'Population 2016 est.']]
    NBA_city_df.rename(columns={'NBA': 'NBA_Team'}, inplace=True)
    NBA_city_df = NBA_city_df[(sport_cities['NBA'] != ' ') &
        (sport_cities['NBA'] != '')]
    NBA_city_df['NBA_Team'] = NBA_city_df['NBA_Team'].str.strip()
      #print(NBA_city_df)
    #print(len(NBA_city_df))
    
    NBA_merge_df = pd.merge(nba_wl_df, NBA_city_df, how = "inner", on = "NBA_Team")
    print(NBA_merge_df)
    
    NBA_agg_func = ["mean"]
    NBA_agg_df = NBA_merge_df.groupby(['Region', 'Population 2016 est.'])['WL'].agg(NBA_agg_func)
    NBA_agg_df = NBA_agg_df.reset_index()
    
    NBA_agg_columns = ['Region','Population by Region', 'Win Loss by Region']
    NBA_agg_df.columns = NBA_agg_columns
    print(NBA_agg_df)
    
    print("***************************************NFL*******************************************************")
    #FootBall Team
    NFL_wl_df = nfl_df[~(nfl_df['team'].str.contains('AFC'))][['team','W', 'W-L%']]
    NFL_wl_df = NFL_wl_df[~(NFL_wl_df['team'].str.contains('NFC'))][['team','W', 'W-L%']]
    NFL_wl_df['team'] = NFL_wl_df['team'].str.replace(r'[+*]$', '', regex = True)
    NFL_wl_df[['Region', 'Team']] = NFL_wl_df['team'].str.rsplit(n=1, expand=True)
    NFL_wl_df['L'] = ( NFL_wl_df['W'].astype(int) - (NFL_wl_df['W-L%'].astype(float) * NFL_wl_df['W'].astype(int)))/NFL_wl_df['W-L%'].astype(float)
    NFL_wl_df['L'] = NFL_wl_df['L'].round(0).astype(int)
    NFL_wl_df['WL'] = NFL_wl_df['W'].astype(int)/( NFL_wl_df['W'].astype(int)+ NFL_wl_df['L'].astype(int))
    NFL_wl_df = NFL_wl_df[['Team','WL','Region']]
    NFL_wl_df['Region'] = NFL_wl_df['Region'].str.replace('New England','Boston')
    NFL_wl_df['Region'] = NFL_wl_df['Region'].str.replace('New York','New York City')
    NFL_wl_df['Region'] = NFL_wl_df['Region'].str.replace('Tenessee','Nashville')
    NFL_wl_df['Region'] = NFL_wl_df['Region'].str.replace('Arizona','Phoenix')
    NFL_wl_df['Region'] = NFL_wl_df['Region'].str.replace('Carolina','Charlotte')
    NFL_wl_df['Region'] = NFL_wl_df['Region'].str.replace('Minnesota','Minneapolis–Saint Paul')
    
    NFL_wl_df['Region'] = NFL_wl_df['Region'].str.strip()
    NFL_wl_df['Team'] = NFL_wl_df['Team'].str.strip()
    NFL_wl_df = NFL_wl_df[['Team', 'WL']]
    NFL_wl_df.rename(columns={'Team': 'NFL_Team'}, inplace=True)
    #print('******************NFL_wl_df******************')
    #print(NFL_wl_df)
    
    ##cities football
    
    #sport_cites = sport_cities[]
    sport_cities['NFL'] = sport_cities['NFL'].str.replace(r'\[note\s+\d+\](?!\w)','', regex = True)
    sport_cities['NFL'] = sport_cities['NFL'].str.replace(r'[^\w\s]+', '', regex = True)
    #print('sport_cites before: ')
    #print(sport_cities)
    
    sport_cities['NFL'] = sport_cities['NFL'].str.replace('Giants Jets', 'Giants, Jets')
    sport_cities['NFL'] = sport_cities['NFL'].str.replace('Rams Chargers', 'Rams, Chargers')
    sport_cities['NFL'] = sport_cities['NFL'].str.replace('49ers Raiders', '49ers,Raiders')
    
    
    NFL_city_df = sport_cities.assign(NFL = sport_cities['NFL'].str.split(',')).explode('NFL')
    NFL_city_df['NFL'] = NFL_city_df['NFL'].str.strip()
    NFL_city_df = NFL_city_df.reset_index()
    NFL_city_df.rename(columns={'NFL': 'NFL_Team'}, inplace=True)
    NFL_city_df = NFL_city_df.reset_index()
    NFL_city_df = NFL_city_df[(NFL_city_df['NFL_Team'] != ' ') & (NFL_city_df['NFL_Team'] != '')]
    #print('***********NFL_city_df**************')
    #print(NFL_city_df)                                   
    NFL_merge_df = pd.merge(NFL_wl_df, NFL_city_df, how = "inner", on = "NFL_Team")
    
    ##initiating the p-value matrix
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    #print(p_values) 
    
    #print("************************************************NFL_merge_df****************************************")
    #print(NFL_merge_df)
    
    NFL_agg_func = ["mean"]
    NFL_agg_df = NFL_merge_df.groupby(['Region', 'Population 2016 est.'])['WL'].agg(NFL_agg_func)
    NFL_agg_df = NFL_agg_df.reset_index()
    
    NFL_agg_columns = ['Region','Population by Region', 'Win Loss by Region']
    NFL_agg_df.columns = NFL_agg_columns
    print('********************NFL_agg_columns********************')
    #print(NFL_agg_df)
    print(len(NFL_agg_df))
    
    ##NFL/NFL-29
    print('*********************************************NFL/NFL-29**********************************************')
    NFL_self_merge = pd.merge(NFL_agg_df, NFL_agg_df, how = 'inner', on = 'Region')
    #print(NFL_self_merge)
    print(len(NFL_self_merge))
    p_NFL_NFL = ttest_rel(NFL_self_merge['Win Loss by Region_x'], NFL_self_merge['Win Loss by Region_y'])
    print(p_NFL_NFL.pvalue)
   
    p_values.loc['NFL','NFL'] = p_NFL_NFL.pvalue
    print(ttest_rel(NFL_self_merge['Win Loss by Region_x'], NFL_self_merge['Win Loss by Region_y']))
    print(ttest_ind(NFL_self_merge['Win Loss by Region_x'], NFL_self_merge['Win Loss by Region_y']))
    #print(p_values)
    ##NFL/NBA-19
    print('*********************************************NFL/NBA-19**********************************************')
    NFL_NBA_merge = pd.merge(NFL_agg_df, NBA_agg_df, how = 'inner', on = 'Region')
    #print(NFL_NBA_merge)
    print(len(NFL_NBA_merge))
    p_NFL_NBA = ttest_rel(NFL_NBA_merge['Win Loss by Region_x'], NFL_NBA_merge['Win Loss by Region_y'])
    print(p_NFL_NBA.pvalue)
    
    p_values.loc['NFL','NBA'] = p_NFL_NBA.pvalue
    print(ttest_rel(NFL_NBA_merge['Win Loss by Region_x'], NFL_NBA_merge['Win Loss by Region_y']))
    print(ttest_ind(NFL_NBA_merge['Win Loss by Region_x'], NFL_NBA_merge['Win Loss by Region_y']))
    #print(p_values)
    
    #NFL/NHL-17
    print('*********************************************NFL/NHL-17**********************************************')
    NFL_NHL_merge = pd.merge(NFL_agg_df, nhl_agg_df, how = 'inner', on = 'Region')
    #print(NFL_NHL_merge)
    print(len(NFL_NHL_merge))
    p_NFL_NHL = ttest_rel(NFL_NHL_merge['Win Loss by Region_x'], NFL_NHL_merge['Win Loss by Region_y'])
    print(p_NFL_NHL.pvalue)
    
    p_values.loc['NFL', 'NHL'] = p_NFL_NHL.pvalue
    print(ttest_rel(NFL_NHL_merge['Win Loss by Region_x'], NFL_NHL_merge['Win Loss by Region_y']))
    print(ttest_ind(NFL_NHL_merge['Win Loss by Region_x'], NFL_NHL_merge['Win Loss by Region_y']))
    #print(p_values)
    
    #NFL/MLB-22
    print('*********************************************NFL/MLB-22**********************************************')
    NFL_MLB_merge = pd.merge(NFL_agg_df, mlb_agg_df , how = 'inner', on = 'Region')
    #print(NFL_MLB_merge)
    print(len(NFL_MLB_merge))
    p_NFL_MLB = ttest_rel(NFL_MLB_merge['Win Loss by Region_x'], NFL_MLB_merge['Win Loss by Region_y'])
    print(p_NFL_MLB.pvalue)
    
    p_values.loc['NFL','MLB'] = p_NFL_MLB.pvalue
    print(ttest_rel(NFL_MLB_merge['Win Loss by Region_x'], NFL_MLB_merge['Win Loss by Region_y']))
    print(ttest_ind(NFL_MLB_merge['Win Loss by Region_x'], NFL_MLB_merge['Win Loss by Region_y']))
    #print(p_values)
    
    ## ### NBA/NFL 19
    print('*********************************************NBA/NFL-19**********************************************')
    #NFL_NBA_merge = pd.merge(NFL_agg_df, NBA_agg_df, how = 'inner', on = 'Region')
    NBA_NFL_merge = pd.merge(NBA_agg_df, NFL_agg_df, how = 'inner', on = 'Region')
    ##print(NFL_NBA_merge)
    #print(len(NFL_NBA_merge))
    #print(NBA_NFL_merge)
    print(len(NBA_NFL_merge))
    
    #p_NBA_NFL = ttest_rel(NFL_NBA_merge['Win Loss by Region_y'], NFL_NBA_merge['Win Loss by Region_x'])
    p_NBA_NFL = ttest_rel(NBA_NFL_merge['Win Loss by Region_x'], NBA_NFL_merge['Win Loss by Region_y'])
    #print(p_NBA_NFL.pvalue)
    
    p_values.loc['NBA','NFL'] = p_NBA_NFL.pvalue
    print(ttest_rel(NBA_NFL_merge['Win Loss by Region_x'], NBA_NFL_merge['Win Loss by Region_y']))
    print(ttest_ind(NBA_NFL_merge['Win Loss by Region_x'], NBA_NFL_merge['Win Loss by Region_y']))
    #print(p_values) 
    
    ### NBA/NBA 28
    print('*********************************************NBA/NBA-28********************************************************')
    self_join_result_nba = pd.merge(NBA_agg_df, NBA_agg_df, on = 'Region')
    #print(self_join_result_nba)
    print(len(self_join_result_nba))
    
    p_NBA_NBA = ttest_rel(self_join_result_nba['Win Loss by Region_x'], self_join_result_nba['Win Loss by Region_y']) 
    
    p_values.loc['NBA', 'NBA'] = p_NBA_NBA.pvalue
    print(ttest_rel(self_join_result_nba['Win Loss by Region_x'], self_join_result_nba['Win Loss by Region_y']))
    print(ttest_ind(self_join_result_nba['Win Loss by Region_x'], self_join_result_nba['Win Loss by Region_y']))
    #End NBA
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    #print(p_values)
    
    ### NBA/NHL 14
    print('*********************************************NBA/NHL-14********************************************************')
    NBA_NHL_merge = pd.merge(NBA_agg_df, nhl_agg_df, how='inner', on='Region')
    #print(NBA_NHL_merge)
    print(len(NBA_NHL_merge))
    p_NBA_NHL = ttest_rel(NBA_NHL_merge['Win Loss by Region_x'],NBA_NHL_merge['Win Loss by Region_y'])
    
    p_values.loc['NBA','NHL'] = p_NBA_NHL.pvalue
    print(ttest_rel(NBA_NHL_merge['Win Loss by Region_x'], NBA_NHL_merge['Win Loss by Region_y']))
    print(ttest_ind(NBA_NHL_merge['Win Loss by Region_x'], NBA_NHL_merge['Win Loss by Region_y']))
    
    #print(p_values)
    ##NBA/NHL#
    
### NBA/MLB 18
    print('*********************************************NBA/MLB-18********************************************************')
    NBA_MLB_merge = pd.merge(NBA_agg_df, mlb_agg_df, how='inner', on= 'Region')
    #print(NBA_MLB_merge)
    print(len(NBA_MLB_merge))
    p_NBA_MLB = ttest_rel(NBA_MLB_merge['Win Loss by Region_x'], NBA_MLB_merge['Win Loss by Region_y'])
    
    p_values.loc['NBA', 'MLB']= p_NBA_MLB.pvalue
    print(ttest_rel(NBA_MLB_merge['Win Loss by Region_x'], NBA_MLB_merge['Win Loss by Region_y']))
    print(ttest_ind(NBA_MLB_merge['Win Loss by Region_x'], NBA_MLB_merge['Win Loss by Region_y']))
    
    #print(p_values)
    
    ### NHL/NFL 17
    print('*********************************************NHL/NFL-17**********************************************')
    NFL_NHL_merge = pd.merge(NFL_agg_df, nhl_agg_df, how = 'inner', on = 'Region')
    #print(NFL_NHL_merge)
    print(len(NFL_NHL_merge))
    #p_NFL_NHL = ttest_rel(NFL_NHL_merge['Win Loss by Region_x'], NFL_NHL_merge['Win Loss by Region_y'])
    p_NHL_NFL = ttest_rel(NFL_NHL_merge['Win Loss by Region_y'], NFL_NHL_merge['Win Loss by Region_x'])
    print(p_NHL_NFL.pvalue)
    
    p_values.loc['NHL', 'NFL'] = p_NHL_NFL.pvalue
    print(ttest_rel(NFL_NHL_merge['Win Loss by Region_y'], NFL_NHL_merge['Win Loss by Region_x']))
    print(ttest_ind(NFL_NHL_merge['Win Loss by Region_y'], NFL_NHL_merge['Win Loss by Region_x']))
    #print(p_values)
    
     ### NBA/NHL 14
    print('*********************************************NHL/NBA-14***********************************************************')
    NBA_NHL_merge = pd.merge(NBA_agg_df, nhl_agg_df, how='inner', on='Region')
    #print(NBA_NHL_merge)
    print(len(NBA_NHL_merge))
    #p_NBA_NHL = ttest_rel(NBA_NHL_merge['Win Loss by Region_x'],NBA_NHL_merge['Win Loss by Region_y'])
    p_NHL_NBA = ttest_rel(NBA_NHL_merge['Win Loss by Region_y'],NBA_NHL_merge['Win Loss by Region_x'])
    
    p_values.loc['NHL', 'NBA'] = p_NBA_NHL.pvalue
    print(ttest_rel(NBA_NHL_merge['Win Loss by Region_y'], NBA_NHL_merge['Win Loss by Region_x']))
    print(ttest_ind(NBA_NHL_merge['Win Loss by Region_y'], NBA_NHL_merge['Win Loss by Region_x']))
    
    #print(p_values)
    ##NBA/NHL#
    
    ### NHL/NHL 28##
    print('**************************************************NHL/NHL-28*******************************************')
    self_join_result_nhl = pd.merge(nhl_agg_df, nhl_agg_df, on = 'Region')
    #print(self_join_result_nhl)
    print(len(self_join_result_nhl))
    
    p_NHL_NHL = ttest_rel(self_join_result_nhl['Win Loss by Region_x'], self_join_result_nhl['Win Loss by Region_y'])
    
    p_values.loc['NHL','NHL'] = p_NHL_NHL.pvalue
    print(ttest_rel(self_join_result_nhl['Win Loss by Region_x'], self_join_result_nhl['Win Loss by Region_y']))
    print(ttest_ind(self_join_result_nhl['Win Loss by Region_x'], self_join_result_nhl['Win Loss by Region_y']))
    
    #print(sport_cities)
    #print(sport_cities.columns)
    #print(p_values)
    
    print('****************************************************NHL/MLB-17******************************************')
    ###MLB/NHL
    ### MLB/NHL 17
    MLB_NHL_merge = pd.merge(mlb_agg_df, nhl_agg_df, how ='inner', on='Region')
    #print(MLB_NHL_merge)
    print(len(MLB_NHL_merge))
    
    p_NHL_MLB = ttest_rel(MLB_NHL_merge['Win Loss by Region_y'], MLB_NHL_merge['Win Loss by Region_x'])
    
    p_values.loc['NHL','MLB'] = p_NHL_MLB.pvalue
    print(ttest_rel(MLB_NHL_merge['Win Loss by Region_y'], MLB_NHL_merge['Win Loss by Region_x']))
    print(ttest_ind(MLB_NHL_merge['Win Loss by Region_y'], MLB_NHL_merge['Win Loss by Region_x']))
    #print(p_values)
    
    #NFL/MLB-22
    print('*********************************************MLB/NFL-22*************************************************')
    NFL_MLB_merge = pd.merge(NFL_agg_df, mlb_agg_df , how = 'inner', on = 'Region')
    #print(NFL_MLB_merge)
    print(len(NFL_MLB_merge))
    p_MLB_NFL = ttest_rel(NFL_MLB_merge['Win Loss by Region_y'], NFL_MLB_merge['Win Loss by Region_x'])
    print(p_MLB_NFL.pvalue)
    
    p_values.loc['MLB','NFL'] = p_MLB_NFL.pvalue
    print(ttest_rel(NFL_MLB_merge['Win Loss by Region_y'], NFL_MLB_merge['Win Loss by Region_x']))
    print(ttest_ind(NFL_MLB_merge['Win Loss by Region_y'], NFL_MLB_merge['Win Loss by Region_x']))
    #print(p_values)
    
    print('*********************************************MLB/NBA-18**************************************************')
    ##MLB/NBA-18
    MLB_NBA_merge = pd.merge(mlb_agg_df, NBA_agg_df, how='inner', on = 'Region')
    #print(MLB_NBA_merge)
    print(len(MLB_NBA_merge))
    p_MLB_NBA = ttest_rel(MLB_NBA_merge['Win Loss by Region_x'], MLB_NBA_merge['Win Loss by Region_y'])
    print(p_MLB_NBA.pvalue)
    
    p_values.loc['MLB','NBA'] = p_MLB_NBA.pvalue
    print(ttest_rel(MLB_NBA_merge['Win Loss by Region_x'], MLB_NBA_merge['Win Loss by Region_y']))
    ##NBA/MLB#
    #print(p_values)
    print('****************************************************MLB/NHL-17****************************************')
    ###MLB/NHL
    ### MLB/NHL 17
    MLB_NHL_merge = pd.merge(mlb_agg_df, nhl_agg_df, how ='inner', on='Region')
    #print(MLB_NHL_merge)
    print(len(MLB_NHL_merge))
    p_MLB_NHL = ttest_rel(MLB_NHL_merge['Win Loss by Region_x'], MLB_NHL_merge['Win Loss by Region_y'])
    #print(p_MLB_NHL.pvalue)
    
    p_values.loc['MLB','NHL'] = p_MLB_NHL.pvalue
    print(ttest_rel(MLB_NHL_merge['Win Loss by Region_x'], MLB_NHL_merge['Win Loss by Region_y']))
    print(ttest_ind(MLB_NHL_merge['Win Loss by Region_x'], MLB_NHL_merge['Win Loss by Region_y']))
   
    #print(p_values)
    print('****************************************************MLB/MLB-26******************************************')
    self_join_result_mlb = pd.merge(mlb_agg_df, mlb_agg_df, on = 'Region')
    #print(self_join_result_mlb)
    print(len(self_join_result_mlb))
    p_MLB_MLB = ttest_rel(self_join_result_mlb['Win Loss by Region_x'], self_join_result_mlb['Win Loss by Region_y'])
    print(p_MLB_MLB)
    
    p_values.loc['MLB','MLB']
    print(ttest_rel(self_join_result_mlb['Win Loss by Region_x'], self_join_result_mlb['Win Loss by Region_y']))
    print(ttest_ind(self_join_result_mlb['Win Loss by Region_x'], self_join_result_mlb['Win Loss by Region_y']))
    print(p_values)
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
    
    ### NFL/NFL 29
    ### NFL/NBA 19
    ### NFL/NHL 17
    ### NFL/MLB 22
    ### NBA/NFL 19
    ### NBA/NBA 28##
    ### NBA/NHL 14##
    ### NBA/MLB 18##
    ### NHL/NFL 17
    ### NHL/NBA 14
    ### NHL/NHL 28##
    ### NHL/MLB 17##
    ### MLB/NFL 22
    ### MLB/NBA 18
    ### MLB/NHL 17
    ### MLB/MLB 26
sports_team_performance()  
    

*****************************************************************************************************
*********************************************MLB*****************************************************
                                                 mean
Region                 Population 2016 est.          
Atlanta                5789700               0.555556
Baltimore              2798886               0.290123
Boston                 4794447               0.666667
Chicago                9512999               0.482769
Cincinnati             2165139               0.413580
Cleveland              2055612               0.561728
Dallas–Fort Worth      7233323               0.413580
Denver                 2853077               0.558282
Detroit                4297617               0.395062
Houston                6772470               0.635802
Kansas City            2104509               0.358025
Los Angeles            13310447              0.529122
Miami–Fort Lauderdale  6066387          

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
